In [6]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 5.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 7.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 7.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 8.1 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 8.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 8.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 8.0 MB/s eta 0:00:

In [18]:
!pip install keras tensorflow
!pip install tensorflow

In [8]:
!pip install pandas
!pip install openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 8.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 7.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 7.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 6.0 MB/s eta 0:00:0000:01


In [71]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from transformers import BertTokenizer, BertModel


df = pd.read_excel(r"C:\Users\kiwizcloud\내 드라이브\Dev\Book_data_drop.xlsx")

In [72]:
#GPU 사용
import torch
torch.cuda.is_available() #True 반환하면 GPU 사용가능 상태 -> 다음 코드 실행

True

In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu') #Cuda(GPU)사용 가능하면 GPU사용, 아니면 CPU사용

# 1. 분류 모델 생성
### (설명 + 목차 모두 사용)

In [79]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np


# 데이터 로드
df = pd.read_excel(r"C:\Users\kiwizcloud\내 드라이브\Dev\Book_data_processed.xlsx")
df['introduction'] = df['introduction'].fillna('')
df['contents'] = df['contents'].fillna('')
df['combined_text'] = df['introduction'] + ' ' + df['contents']

# 가중치 설정
title_weight = 1.2  # 상품명 가중치
author_weight = 1.0  # 작가 가중치
category_weight = 1.0  # 대분류 가중치
description_weight = 1.3  # 설명 가중치

# 상품명 벡터화
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['title'])
weighted_tfidf_matrix = title_weight * tfidf_matrix  # 가중치 부여

# 작가 정보 벡터화
author_vectorizer = TfidfVectorizer()
author_matrix = author_vectorizer.fit_transform(df['author'])
weighted_author_matrix = author_weight * author_matrix  # 가중치 부여

# 대분류 정보 원-핫 인코딩
category_encoder = OneHotEncoder()
category_matrix = category_encoder.fit_transform(df[['category2']])
weighted_category_matrix = category_weight * category_matrix  # 가중치 부여


# BERT 모델 로드
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.to(device)

# BERT를 사용하여 설명 정보 벡터화
def bert_vectorize(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return embeddings

# 결합 텍스트 벡터화
description_vectors = df['combined_text'].apply(bert_vectorize)
description_matrix = np.vstack(description_vectors.values)
weighted_description_matrix = description_weight * description_matrix

# 상품명, 작가, 대분류, 설명 정보를 결합하여 최종 행렬 생성
final_matrix = hstack((weighted_tfidf_matrix, weighted_author_matrix, weighted_category_matrix, weighted_description_matrix)).toarray()

## isbn추출 및 h5파일 저장

In [99]:
df['isbn'].astype('int64')

0       9791198517425
1       9791193937068
2       9791198564139
3       9791130653150
4       9791191056372
            ...      
2003    9791192987965
2004    9791191905755
2005    9791193205235
2006    9788931475388
2007    9788965020110
Name: isbn, Length: 2008, dtype: int64

In [103]:
#isbn값 추출
isbn_list = df['isbn'].astype('int64').tolist() #그냥 int형으로 저장하게 되면 오버플로우 발생해 isbn데이터 값이 깨짐

# 최종 행렬을 DataFrame으로 변환하고 ISBN을 인덱스로 설정
final_matrix_df = pd.DataFrame(final_matrix, index=isbn_list)

In [102]:
final_matrix_df

,0,1,2,3,4,5,6,7,8,9,...,6393,6394,6395,6396,6397,6398,6399,6400,6401,6402
9791198517425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.113871,0.351395,-0.529766,-1.277292,0.035310,-0.353148,0.136427,0.369242,0.488639,0.091147
9791193937068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.171356,0.283931,-0.467369,-1.381355,0.016641,-0.456644,0.414899,0.377052,0.349531,0.000041
9791198564139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.223073,0.292066,-0.450192,-1.383421,0.091756,-0.432998,0.323660,0.494477,0.433761,-0.062013
9791130653150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.145764,0.146797,-0.370518,-1.216604,0.041915,-0.262993,0.266227,0.360382,0.339611,0.153659
9791191056372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.131984,0.353868,-0.609061,-1.296685,0.068275,-0.429050,0.226057,0.341837,0.404175,0.078428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9791192987965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.083384,0.200328,-0.641535,-1.359036,0.059749,-0.266535,0.257491,0.582767,0.586813,-0.209493
9791191905755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.170657,0.070015,-0.544429,-1.290481,0.027342,-0.342865,0.277322,0.496223,0.491489,-0.227157
9791193205235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.161930,0.261295,-0.358940,-1.311567,0.058679,-0.521033,0.388074,0.677100,0.397947,-0.060805
9788931475388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.202967,0.077903,-0.606460,-1.295104,-0.018945,-0.188948,0.290815,0.280895,0.356512,-0.113556


In [104]:
import pandas as pd
import numpy as np
import h5py

# 최종 행렬을 h5 파일로 저장
with h5py.File('final_matrix_isbn.h5', 'w') as h5_file:
    h5_file.create_dataset('final_matrix', data=final_matrix_df.values)
    h5_file.create_dataset('isbn_list', data=np.array(isbn_list), dtype=np.int64)
    # h5py에서 데이터셋을 생성할 때는 데이터 형태를 명시적으로 지정해야함(int형)


### 저장내용 확인

In [105]:
with h5py.File(r"C:\Users\kiwizcloud\내 드라이브\Dev\final_matrix_isbn.h5", 'r') as h5_file:
    print(list(h5_file.keys()))

['final_matrix', 'isbn_list']


In [106]:
with h5py.File(r"C:\Users\kiwizcloud\내 드라이브\Dev\final_matrix_isbn.h5", 'r') as h5_file:
    isbn_list = h5_file['isbn_list'][:]
    print(isbn_list)

[9791198517425 9791193937068 9791198564139 ... 9791193205235 9788931475388
 9788965020110]


# 2. 추천과정

In [107]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

# ISBN13 값으로 책 찾기
def find_book_by_isbn13(isbn13):
    # ISBN13 값을 정수로 변환
    isbn13 = int(isbn13)
    book = df[df['isbn'] == isbn13]
    if len(book) > 0:
        print(book.index[0])
        return book.index[0]
    else:
        return None

# 유사도 계산 함수
def get_top_similar_books(book_index, top_n=5):
    # 코사인 유사도 계산
    cosine_sim = cosine_similarity(final_matrix)

    # 유클리디안 거리 계산
    euclidean_dist = euclidean_distances(final_matrix)

    # 거리 행렬을 유사도 행렬로 변환
    euclidean_sim = 1 / (1 + euclidean_dist)

    sim_scores = list(enumerate(cosine_sim[book_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i for i, _ in sim_scores[1:top_n+1]]
    top_books = df.iloc[top_indices][['isbn', 'title', 'author', 'category2']]
    return top_books

# 사용자로부터 ISBN13 값 입력 받기
isbn13 = input("찾고자 하는 책의 ISBN13을 입력하세요: ")

# ISBN13 값으로 책 찾기
book_index = find_book_by_isbn13(isbn13)

if book_index is not None:
    # 유사한 책 찾기
    top_similar_books = get_top_similar_books(book_index, top_n=5)
    print(f"'{df.loc[book_index, 'title']}' 책과 유사한 책:")
    print(top_similar_books)
else:
    print("해당 ISBN13을 가진 책이 존재하지 않습니다.")

찾고자 하는 책의 ISBN13을 입력하세요:  9791193926246


1998
'AWS 시스템 개발 스킬업' 책과 유사한 책:
               isbn                 title    author category2
1919  9791192987675    클라우드 네이티브 스프링 인 액션   토마스 비탈레    IT 모바일
2003  9791192987965    러스트 서버, 서비스, 앱 만들기  프라부 에스왈라    IT 모바일
1863  9791169212366  한 권으로 배우는 도커 & 쿠버네티스       장철원    IT 모바일
1846  9791193926208             자바 잘 읽는 법  로렌티우 스필카    IT 모바일
1987  9791190665209   그림으로 공부하는 IT 인프라 구조  야마자키 야스시    IT 모바일


# 3. 북카트 추천부분

In [113]:
# 사용자의 북카트 정보 저장
user_cart = []

# 북카트에 책 추가하는 함수
def add_to_cart(isbn13):
    book_index = find_book_by_isbn13(isbn13)
    if book_index is not None:
        user_cart.append(book_index)
        print(f"'{df.loc[book_index, 'title']}' 책이 북카트에 추가되었습니다.")
    else:
        print("해당 ISBN13을 가진 책이 존재하지 않습니다.")

# 북카트를 기반으로 유사한 책 추천하는 함수
def recommend_books_from_cart(top_n=5):
    if len(user_cart) == 0:
        print("북카트가 비어있습니다.")
        return

    # 북카트에 담긴 책들의 유사도 점수 합산
    cart_scores = np.zeros(len(df))
    for book_index in user_cart:
        cart_scores += cosine_similarity(final_matrix[book_index].reshape(1, -1), final_matrix)[0]

    # 유사도 점수 기준으로 정렬
    sim_scores = list(enumerate(cart_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 북카트에 담긴 책들 제외하고 상위 N개 책 추천
    top_indices = [i for i, _ in sim_scores if i not in user_cart][:top_n]
    top_books = df.iloc[top_indices][['isbn', 'title', 'author', 'category2']]
    return top_books

# 사용자로부터 ISBN13 값 입력 받기
while True:
    isbn13 = input("북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q): ")
    if isbn13 == 'q':
        break
    add_to_cart(isbn13)

# 북카트 기반 책 추천
recommended_books = recommend_books_from_cart(top_n=5)
print("북카트를 기반으로 한 추천 책:")
print(recommended_books)

북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  9791161726663


795
'한 컷 쏙 과학사' 책이 북카트에 추가되었습니다.


북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  9791188829408


1072
'특별하게 제주' 책이 북카트에 추가되었습니다.


북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  9791190073240


...

1579
'왓칭 Watching' 책이 북카트에 추가되었습니다.


북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  9791192987910


1918
'초자동화 시대가 온다' 책이 북카트에 추가되었습니다.


북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  q


북카트를 기반으로 한 추천 책:
               isbn          title   author category2
1600  9791165219550        미적분의 쓸모      한화택      자연과학
970   9791190073318    에이든 홍콩 여행지도    타블라라사     건강 취미
1478  9791192625294        요약이 힘이다  사이토 다카시      자기계발
1147  9791140706631  무작정 따라하기 싱가포르      전상현     건강 취미
1258  9788998075088    상처 받지 않는 영혼   마이클 싱어        인문


## np.mean사용

In [117]:
# 사용자의 북카트 정보 저장
user_cart = []

# 북카트에 책 추가하는 함수
def add_to_cart(isbn13):
    book_index = find_book_by_isbn13(isbn13)
    if book_index is not None:
        user_cart.append(book_index)
        print(f"'{df.loc[book_index, 'title']}' 책이 북카트에 추가되었습니다.")
    else:
        print("해당 ISBN13을 가진 책이 존재하지 않습니다.")

# 북카트를 기반으로 유사한 책 추천하는 함수
def recommend_books_from_cart(top_n=5):
    if len(user_cart) == 0:
        print("북카트가 비어있습니다.")
        return

    individual_scores = []
    for book_index in user_cart:
        scores = cosine_similarity(final_matrix[book_index].reshape(1, -1), final_matrix)[0]
        individual_scores.append(scores)

    # 개별 유사도 점수 합산 방식 (예: 최대값 사용 -> 결과가 별로여서 평균값 사용)
    combined_scores = np.mean(individual_scores, axis=0)

    # 유사도 점수 기준으로 정렬
    sim_scores = list(enumerate(combined_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 북카트에 담긴 책들 제외하고 상위 N개 책 추천
    top_indices = [i for i, _ in sim_scores if i not in user_cart][:top_n]
    top_books = df.iloc[top_indices][['isbn', 'title', 'author', 'category2']]
    return top_books

# 사용자로부터 ISBN13 값 입력 받기
while True:
    isbn13 = input("북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q): ")
    if isbn13 == 'q':
        break
    add_to_cart(isbn13)

# 북카트 기반 책 추천
recommended_books = recommend_books_from_cart(top_n=5)
print("북카트를 기반으로 한 추천 책:")
print(recommended_books)

북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  9788901260679


91
'넛지: 파이널 에디션' 책이 북카트에 추가되었습니다.


북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  9791188279807


109
'볼린저 밴드 투자기법' 책이 북카트에 추가되었습니다.


북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  9791171170418


116
'일론 머스크' 책이 북카트에 추가되었습니다.


북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  9791193059289


463
'교실에서 바로 통하는 배움중심수업 에듀테크와 AI로 확!잡자' 책이 북카트에 추가되었습니다.


북카트에 추가할 책의 ISBN13을 입력하세요 (종료: q):  q


북카트를 기반으로 한 추천 책:
              isbn                      title     author category2
215  9788998075200                     사업의 철학     마이클 거버     경제 경영
4    9791191056372  돈의 심리학 (30만 부 기념 스페셜 에디션)     모건 하우절     경제 경영
107  9788960519831                        칩 워     크리스 밀러     경제 경영
17   9791165349707                    AI 사피엔스        최재붕     경제 경영
211  9791155816066                  AI 이후의 세계  헨리 A. 키신저     경제 경영
